In [25]:
from imdb import Cinemagoer
import pandas as pd

In [ ]:
# creates instance of cinemagoer
ia = Cinemagoer()

## Collect Grey's Anatomy Episode List

In [68]:
# get first grey's anatomy episode from manually collected id and adds it to a list to collect the rest of them
greys_prem = ia.get_movie('0592902')

episode_list = [greys_prem]

'0592923'

In [70]:
# Helper method for populating episode data tables (dodges IMDB API bug that doesn't show episode lists)
def get_next_id(prev_ep):
    return ia.get_movie(prev_ep['next episode'])

In [336]:
# This times out a lot so this helps prevent restarting
episode_list = episode_list
error_count = 0

# makes api requests to collect episodes as Movie Items at ~25eps per minute (will be rate limited at 1000 collections per day) 
while (error_count < 10):
    try: 
        episode_list.append(get_next_id(episode_list[len(episode_list) - 1]))
        print(len(episode_list), "episodes completed")
    except TimeoutError:
        print("Timeout error occurred: TimeoutError exception")
    except Exception as e:
        print(e, type(e))
        break
        

'next episode' <class 'KeyError'> whyyy


In [84]:
# saves updated list just in case
save_list = episode_list

In [302]:
# checks collected list
save_list, len(save_list), save_list[0]['number of episodes']

([<Movie id:0592902[http] title:_"Grey's Anatomy" A Hard Day's Night (2005)_>,
  <Movie id:0592923[http] title:_"Grey's Anatomy" The First Cut Is the Deepest (2005)_>,
  <Movie id:0592926[http] title:_"Grey's Anatomy" Winning a Battle, Losing the War (2005)_>,
  <Movie id:0592915[http] title:_"Grey's Anatomy" No Man's Land (2005)_>,
  <Movie id:0592919[http] title:_"Grey's Anatomy" Shake Your Groove Thing (2005)_>,
  <Movie id:0592909[http] title:_"Grey's Anatomy" If Tomorrow Never Comes (2005)_>,
  <Movie id:0592924[http] title:_"Grey's Anatomy" The Self-Destruct Button (2005)_>,
  <Movie id:0592918[http] title:_"Grey's Anatomy" Save Me (2005)_>,
  <Movie id:0592925[http] title:_"Grey's Anatomy" Who's Zoomin' Who? (2005)_>,
  <Movie id:0592917[http] title:_"Grey's Anatomy" Raindrops Keep Falling on My Head (2005)_>,
  <Movie id:0592907[http] title:_"Grey's Anatomy" Enough Is Enough (2005)_>,
  <Movie id:0592913[http] title:_"Grey's Anatomy" Make Me Lose Control (2005)_>,
  <Movie id:0

In [319]:
# helper method for convert to dict, collects writer info and returns a list of dicts with writer info
def extract_writers(writer_list):
    writers_info = []
    for writer in writer_list:
        try:
            if writer.personID not in (writer['writer_id'] for writer in writers_info):
                writer_info = {'writer_id': writer.personID, 'name': writer['name']}
                writers_info.append(writer_info)
        except:
            pass
    return writers_info


In [277]:
# collects info from episodes as a dict to prep for entry into db
def convert_to_dict(item):
    try:
        writers = extract_writers(item['writers'])
    except KeyError:
        writers = None

    try:
        description = item['plot'][-1]
    except (KeyError, IndexError):
        description = None
    
    return {
        'ep_id': item['imdbID'],
        'title': item['title'],
        'season': item['season'],
        'ep_num': item['episode'],
        'air_date': item['original air date'],
        'description': description,
        'imdb_rate': item['rating'],
        'writers': writers
    }

In [297]:
# collects converted items as dicts in a list
ep_info = [convert_to_dict(ep) for ep in save_list]

In [298]:
# creates dataframe of episodes
greys_eps_df = pd.DataFrame(ep_info).set_index('ep_id')

# separates writers from episode table
exploded_writers_df = greys_eps_df.explode('writers')
writing_creds = pd.DataFrame(exploded_writers_df['writers'].tolist(), index=exploded_writers_df.index)
greys_eps_df.drop(['writers'], inplace=True, axis=1)

# collects writer info into a writer table
writers_df = pd.DataFrame(writing_creds.name.unique(), index=writing_creds.writer_id.unique()).rename(columns={0:'name'})
writers_df.index.name = 'writer_id'

# collects episode specific writer associations (ep_id and writer_id) into writer_creds table
writing_creds = writing_creds.drop(['name'], axis=1)

writing_creds


,writer_id
ep_id,
0592902,0722274
0592923,0722274
0592926,0722274
0592915,0722274
0592919,0722274
...,...
27440369,8108153
27435239,0722274
27441536,0722274


In [300]:
# adds show_id column (to be converted to season_id)
greys_eps_df['show_id'] = '413573'
greys_eps_df

,title,season,ep_num,air_date,description,imdb_rate,show_id
ep_id,,,,,,,
0592902,A Hard Day's Night,1,1,27 Mar 2005,Meet Meredith Grey: daughter of famous surgeon...,8.3,413573
0592923,The First Cut Is the Deepest,1,2,03 Apr 2005,The surgical staff tries to be optimistic as a...,8.1,413573
0592926,"Winning a Battle, Losing the War",1,3,10 Apr 2005,"A bike race injures several people, one of who...",8.0,413573
0592915,No Man's Land,1,4,17 Apr 2005,George feels less masculine when Izzie and Mer...,8.0,413573
0592919,Shake Your Groove Thing,1,5,24 Apr 2005,Meredith realizes that she must take on a lot ...,8.1,413573
...,...,...,...,...,...,...,...
27440369,Gunpowder and Lead,19,16,20 Apr 2023,Amelia takes her personal problems out on her ...,7.4,413573
27435239,Come Fly With Me,19,17,04 May 2023,Teddy calls an emergency meeting to discuss th...,7.7,413573
27441536,Ready to Run,19,18,11 May 2023,Richard and Teddy make an exciting announcemen...,7.5,413573


## Functions to use
Based on above work

In [342]:
def collect_episodes(prem_id):
    episode_list = [ia.get_movie(prem_id)]
    save_list = episode_list
    

    # makes api requests to collect episodes as Movie Items at ~25eps per minute (will be rate limited at 1000 collections per day) 
    error_count = 0 # prevents using up api requests on broken code, gives a chance for it to timeout sometimes
    while (error_count < 20):
        try: 
            episode_list.append(get_next_id(episode_list[len(episode_list) - 1]))
            print(len(episode_list), "episodes completed")
        except TimeoutError:
            print("Timeout error occurred")
        except Exception as e:
            error_count += 1
            print(e)
            save_list = episode_list
    
    print("Collected ", len(episode_list), " episodes. Does this look right?")
    if len(episode_list) == episode_list[0]['number of episodes']:
        print("There are the right number of episodes!")
    else:
        print("Check the number of episodes collected...")
    print(episode_list)
    return episode_list

In [304]:
def add_eps_to_df(episodes, show_id):
    ep_info = [convert_to_dict(ep) for ep in episodes]
    
    # creates dataframe of episodes
    show_episodes_df = pd.DataFrame(ep_info).set_index('ep_id')

    # separates writers from episode table
    exploded_writers_df = show_episodes_df.explode('writers')
    writing_creds = pd.DataFrame(exploded_writers_df['writers'].tolist(), index=exploded_writers_df.index)
    show_episodes_df.drop(['writers'], inplace=True, axis=1)

    # collects writer info into a writer table
    writers_df = pd.DataFrame(writing_creds.name.unique(), index=writing_creds.writer_id.unique()).rename(columns={0:'name'})
    writers_df.index.name = 'writer_id'

    # collects episode specific writer associations (ep_id and writer_id) into writer_creds table
    writing_creds = writing_creds.drop(['name'], axis=1)

    show_episodes_df['show_id'] = show_id

    return show_episodes_df, writers_df, writing_creds


## Collecting House Episodes

In [ ]:
premiere_id = '0606035'
house_episode_list = collect_episodes(premiere_id)

In [320]:
house_eps_df, house_writers_df, house_writing_creds = add_eps_to_df(house_episode_list, '0412142')

In [321]:
house_eps_df

,title,season,ep_num,air_date,description,imdb_rate,show_id
ep_id,,,,,,,
0606035,Pilot,1,1,16 Nov 2004,House and his team have a case with a 29-year-...,8.5,0412142
0606034,Paternity,1,2,23 Nov 2004,A 16-year old lacrosse player starts suffering...,8.1,0412142
0606033,Occam's Razor,1,3,30 Nov 2004,Student Brandon Merrell (22) is rushed to ER w...,8.1,0412142
0606030,Maternity,1,4,07 Dec 2004,Two babies in separate delivery rooms delivere...,8.3,0412142
0606018,Damned If You Do,1,5,14 Dec 2004,A nun arrives at the clinic with what appears ...,8.0,0412142
...,...,...,...,...,...,...,...
2121961,Body and Soul,8,18,23 Apr 2012,The team analyzes the significance of dreams.,7.9,0412142
2121962,The C Word,8,19,30 Apr 2012,"When the team takes on the case of Emily, a si...",9.0,0412142
2121963,Post Mortem,8,20,07 May 2012,The team treats a fellow doctor wary of their ...,8.7,0412142


In [322]:
house_writers_df

,name
writer_id,
0794914,David Shore
0438432,Lawrence Kaplow
0086694,Peter Blake
0153384,Sara B. Cooper
0542535,John Mankiewicz
0602981,Thomas L. Moran
0936976,Matt Witten
1844375,David Foster
1403671,Joel Anderson Thompson


In [323]:
house_writing_creds

,writer_id
ep_id,
0606035,0794914
0606034,0794914
0606034,0438432
0606033,0794914
0606030,0794914
...,...
2121964,1068424
2121964,1844375
2121965,0794914


## Collecting ER Episodes

In [337]:
premiere_id = '0295222'
er_episode_list = collect_episodes(premiere_id)

2 episodes completed
3 episodes completed
4 episodes completed
5 episodes completed
6 episodes completed
7 episodes completed
8 episodes completed
9 episodes completed
10 episodes completed
11 episodes completed
12 episodes completed
13 episodes completed
14 episodes completed
15 episodes completed
16 episodes completed
17 episodes completed
18 episodes completed
19 episodes completed
20 episodes completed
21 episodes completed
22 episodes completed
23 episodes completed
24 episodes completed
25 episodes completed
26 episodes completed
27 episodes completed
28 episodes completed
29 episodes completed
30 episodes completed
31 episodes completed
32 episodes completed
33 episodes completed
34 episodes completed
35 episodes completed
36 episodes completed
37 episodes completed
38 episodes completed
39 episodes completed
40 episodes completed
41 episodes completed
42 episodes completed
43 episodes completed
44 episodes completed
45 episodes completed
46 episodes completed
47 episodes comple

2024-02-10 23:41:29,232 CRITICAL [imdbpy] /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0567987/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
               ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 525, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 634, in http_response
    response = self.parent.error(
     

{'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0567987/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}
72 episodes completed
73 episodes completed
74 episodes completed
75 episodes completed
76 episodes completed
77 episodes completed
78 episodes completed
79 episodes completed
80 episodes completed
81 episodes completed
82 episodes completed
83 episodes completed
84 episodes completed
85 episodes completed
86 episodes completed
87 episodes completed
88 episodes completed
89 episodes completed
90 episodes completed
91 episodes completed
92 episodes completed
93 episodes completed
94 episodes completed
95 episodes completed
96 episodes completed
97 episodes completed
98 episodes completed
99 episodes completed
100 episodes completed
101 episodes completed
102 episodes completed
103 episodes completed
104 episodes completed
105 episodes completed
106 episodes completed
107 episodes compl

2024-02-10 23:46:55,529 CRITICAL [imdbpy] /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0568055/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
               ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 525, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 634, in http_response
    response = self.parent.error(
     

{'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0568055/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}
192 episodes completed
193 episodes completed
194 episodes completed
195 episodes completed
196 episodes completed
197 episodes completed
198 episodes completed
199 episodes completed
200 episodes completed
201 episodes completed
202 episodes completed
203 episodes completed
204 episodes completed
205 episodes completed
206 episodes completed
207 episodes completed
208 episodes completed
209 episodes completed
210 episodes completed
211 episodes completed
212 episodes completed
213 episodes completed
214 episodes completed
215 episodes completed
216 episodes completed
217 episodes completed
218 episodes completed
219 episodes completed
220 episodes completed
221 episodes completed
222 episodes completed
223 episodes completed
224 episodes completed
225 episodes completed
226 episodes 

2024-02-10 23:49:08,906 CRITICAL [imdbpy] /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0568128/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
               ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 525, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 634, in http_response
    response = self.parent.error(
     

{'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0568128/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}
235 episodes completed
236 episodes completed
237 episodes completed
238 episodes completed
239 episodes completed
240 episodes completed
241 episodes completed
242 episodes completed
243 episodes completed
244 episodes completed
245 episodes completed
246 episodes completed
247 episodes completed
248 episodes completed
249 episodes completed
250 episodes completed
251 episodes completed
252 episodes completed
253 episodes completed
254 episodes completed
255 episodes completed
256 episodes completed
257 episodes completed
258 episodes completed
259 episodes completed
260 episodes completed
261 episodes completed
262 episodes completed
263 episodes completed
264 episodes completed
265 episodes completed
266 episodes completed
267 episodes completed
268 episodes completed
269 episodes 

In [338]:
er_eps_df, er_writers_df, er_writing_creds = add_eps_to_df(er_episode_list, '0108757')

In [339]:
er_eps_df

,title,season,ep_num,air_date,description,imdb_rate,show_id
ep_id,,,,,,,
0295222,24 Hours,1,1,19 Sep 1994,In the introductory episode to the long-runnin...,8.5,0108757
0567951,Day One,1,2,22 Sep 1994,The day kicks off early when Dr. Lewis is woke...,8.0,0108757
0567992,Going Home,1,3,29 Sep 1994,Nurse Hathaway plucks up the courage to go bac...,8.0,0108757
0568004,Hit and Run,1,4,06 Oct 1994,Carter is woken up by Benton and it isn't long...,7.9,0108757
0568018,Into That Good Night,1,5,13 Oct 1994,Ross and Carter are woken up at 16:00 to conti...,7.9,0108757
...,...,...,...,...,...,...,...
1248514,What We Do,15,18,05 Mar 2009,As Archie repaints his apartment to please Cla...,7.9,0108757
1248515,Old Times,15,19,12 Mar 2009,A girl drops off a sick baby at the ER and the...,9.0,0108757
1248517,Shifting Equilibrium,15,20,19 Mar 2009,"On her last day, as she stands before the wall...",7.9,0108757


In [340]:
er_writers_df

,name
writer_id,
0000341,Michael Crichton
0920274,John Wells
0940963,Lydia Woodward
0543355,Paul Manning
0622250,Robert Nathan
0046371,Neal Baer
0313077,Lance Gentile
0827873,Tracey Stern
0282378,Carol Flint


In [341]:
er_writing_creds

,writer_id
ep_id,
0295222,0000341
0567951,0000341
0567951,0920274
0567992,0000341
0567992,0940963
...,...
1248518,0755161
1248518,1557867
1248519,0000341


## Collecting The Good Doctor Episodes

In [343]:
premiere_id = '6886400'
gooddoc_episode_list = collect_episodes(premiere_id)

2 episodes completed
3 episodes completed
4 episodes completed
5 episodes completed
6 episodes completed
7 episodes completed


2024-02-10 23:58:53,349 CRITICAL [imdbpy] /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt7524390/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
               ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 525, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 634, in http_response
    response = self.parent.error(
     

{'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt7524390/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}
8 episodes completed
9 episodes completed
10 episodes completed
11 episodes completed
12 episodes completed
13 episodes completed
14 episodes completed
15 episodes completed
16 episodes completed
17 episodes completed
18 episodes completed
19 episodes completed
20 episodes completed
21 episodes completed
22 episodes completed
23 episodes completed
24 episodes completed
25 episodes completed
26 episodes completed
27 episodes completed
28 episodes completed
29 episodes completed
30 episodes completed
31 episodes completed
32 episodes completed
33 episodes completed
34 episodes completed
35 episodes completed
36 episodes completed


2024-02-11 00:00:44,016 CRITICAL [imdbpy] /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt9739388/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
               ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 525, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 634, in http_response
    response = self.parent.error(
     

{'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt9739388/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}
37 episodes completed
38 episodes completed
39 episodes completed
40 episodes completed
41 episodes completed
42 episodes completed
43 episodes completed
44 episodes completed
45 episodes completed
46 episodes completed
47 episodes completed
48 episodes completed
49 episodes completed
50 episodes completed
51 episodes completed
52 episodes completed
53 episodes completed
54 episodes completed
55 episodes completed
56 episodes completed
57 episodes completed
58 episodes completed
59 episodes completed
60 episodes completed
61 episodes completed
62 episodes completed
63 episodes completed
64 episodes completed
65 episodes completed
66 episodes completed
67 episodes completed
68 episodes completed
69 episodes completed
70 episodes completed
71 episodes completed
72 episodes completed
73 

In [345]:
gooddoc_eps_df, gooddoc_writers_df, gooddoc_writing_creds = add_eps_to_df(gooddoc_episode_list[:-4], '6470478')

In [346]:
gooddoc_eps_df

,title,season,ep_num,air_date,description,imdb_rate,show_id
ep_id,,,,,,,
6886400,Burnt Food,1,1,25 Sep 2017,The president of San Jose St. Bonaventure Hosp...,8.6,6470478
7179006,Mount Rushmore,1,2,02 Oct 2017,When prejudices about his autism force Shaun t...,8.3,6470478
7375334,Oliver,1,3,09 Oct 2017,"Dr Melendez is fed up with Sean, so sends Sean...",7.8,6470478
7375342,Pipes,1,4,16 Oct 2017,Dr. Neil Melendez and the team have to wade th...,8.1,6470478
7375346,Point Three Percent,1,5,23 Oct 2017,"At St. Bonaventure Hospital, Dr. Shaun Murphy ...",8.5,6470478
...,...,...,...,...,...,...,...
27190137,A Blip,6,18,03 Apr 2023,When a patient comes in with persistent and li...,7.8,6470478
27190153,Half Measures,6,19,10 Apr 2023,Drs. Andrews and Lim clash over a patient slit...,7.9,6470478
27190173,Blessed,6,20,17 Apr 2023,Dr. Marcus Andrews introduces some of the team...,7.8,6470478


In [347]:
gooddoc_writers_df

,name
writer_id,
0794914,David Shore
6728876,Park Jae-beom
0497556,Johanna Lee
3634351,David Renaud
2799425,Karen Struck
4259499,Lloyd Gilyard Jr.
0745360,Bill Rotko
0602981,Thomas L. Moran
0395924,David Hoselton


In [348]:
gooddoc_writing_creds

,writer_id
ep_id,
6886400,0794914
7179006,0794914
7179006,6728876
7179006,0497556
7179006,3634351
...,...
27190233,1068424
27190233,2675977
27190233,4784537


## Collecting Scrubs Episodes

In [349]:
premiere_id = '0696640'
scrubs_episode_list = collect_episodes(premiere_id)

2 episodes completed
3 episodes completed
4 episodes completed
5 episodes completed
6 episodes completed
7 episodes completed
8 episodes completed
9 episodes completed
10 episodes completed
11 episodes completed
12 episodes completed
13 episodes completed
14 episodes completed
15 episodes completed
16 episodes completed
17 episodes completed
18 episodes completed
19 episodes completed
20 episodes completed
21 episodes completed
22 episodes completed
23 episodes completed
24 episodes completed
25 episodes completed
26 episodes completed
27 episodes completed
28 episodes completed
29 episodes completed
30 episodes completed
31 episodes completed
32 episodes completed
33 episodes completed
34 episodes completed
35 episodes completed
36 episodes completed
37 episodes completed
38 episodes completed
39 episodes completed
40 episodes completed
41 episodes completed
42 episodes completed
43 episodes completed
44 episodes completed
45 episodes completed
46 episodes completed
47 episodes comple

2024-02-11 00:09:35,541 CRITICAL [imdbpy] /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0696612/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
               ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 525, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 634, in http_response
    response = self.parent.error(
     

{'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0696612/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}
81 episodes completed
82 episodes completed
83 episodes completed
84 episodes completed
85 episodes completed
86 episodes completed
87 episodes completed
88 episodes completed
89 episodes completed
90 episodes completed
91 episodes completed
92 episodes completed
93 episodes completed
94 episodes completed
95 episodes completed
96 episodes completed
97 episodes completed
98 episodes completed
99 episodes completed
100 episodes completed
101 episodes completed
102 episodes completed
103 episodes completed
104 episodes completed
105 episodes completed
106 episodes completed
107 episodes completed
108 episodes completed
109 episodes completed
110 episodes completed
111 episodes completed
112 episodes completed
113 episodes completed
114 episodes completed
115 episodes completed
116 episo

In [350]:
scrubs_eps_df, scrubs_writers_df, scrubs_writing_creds = add_eps_to_df(scrubs_episode_list, '0285403')

In [351]:
scrubs_eps_df

,title,season,ep_num,air_date,description,imdb_rate,show_id
ep_id,,,,,,,
0696640,My First Day,1,1,02 Oct 2001,"John Dorian, or J. D., is feeling on top of th...",8.4,0285403
0696602,My Mentor,1,2,04 Oct 2001,"Sardonic Dr. Cox gets to J.D., who's in desper...",8.0,0285403
0696547,My Best Friend's Mistake,1,3,09 Oct 2001,"J.D, convinced that one of his patients is sic...",7.9,0285403
0696616,My Old Lady,1,4,16 Oct 2001,The whole crew learn their share about death w...,9.1,0285403
0696636,My Two Dads,1,5,23 Oct 2001,"While J.D. continues to misinterpret Dr. Cox, ...",8.1,0285403
...,...,...,...,...,...,...,...
1571102,Our Stuff Gets Real,9,9,12 Jan 2010,J.D. and Elliot's sex life disappears as they ...,6.5,0285403
1571322,Our True Lies,9,10,19 Jan 2010,Dr. Cox tries to find out which member of Lucy...,6.3,0285403
1564292,Our Dear Leaders,9,11,26 Jan 2010,Turk is jealous when he is upstaged by a talen...,5.9,0285403


In [352]:
scrubs_writers_df

,name
writer_id,
0492639,Bill Lawrence
0777372,Mike Schwartz
0918111,Eric Weinberg
0850696,Matt Tarses
0232981,Garrett Donovan
1054832,Neil Goldman
1000302,Gabrielle Allan
1052109,Janae Bakken
0285975,Debra Fordham


In [353]:
scrubs_writing_creds

,writer_id
ep_id,
0696640,0492639
0696602,0492639
0696602,0777372
0696547,0492639
0696547,0918111
...,...
1568222,0492639
1568222,1558835
1568222,1140565


## Consolidate tables

In [354]:
episodes_df = pd.concat([greys_eps_df, house_eps_df, er_eps_df, scrubs_eps_df, gooddoc_eps_df])
writers_df = pd.concat([writers_df, house_writers_df, er_writers_df, scrubs_writers_df, gooddoc_writers_df])
writing_creds_df = pd.concat([writing_creds, house_writing_creds, er_writing_creds, scrubs_writing_creds, gooddoc_writing_creds])

In [360]:
episodes_df[(episodes_df['season'] == 1) & (episodes_df['ep_num'] == 1)]

,title,season,ep_num,air_date,description,imdb_rate,show_id
ep_id,,,,,,,
0592902,A Hard Day's Night,1,1,27 Mar 2005,Meet Meredith Grey: daughter of famous surgeon...,8.3,413573
0606035,Pilot,1,1,16 Nov 2004,House and his team have a case with a 29-year-...,8.5,0412142
0295222,24 Hours,1,1,19 Sep 1994,In the introductory episode to the long-runnin...,8.5,0108757
0696640,My First Day,1,1,02 Oct 2001,"John Dorian, or J. D., is feeling on top of th...",8.4,0285403
6886400,Burnt Food,1,1,25 Sep 2017,The president of San Jose St. Bonaventure Hosp...,8.6,6470478


In [368]:
writers_df

,name
writer_id,
0722274,Shonda Rhimes
1586791,Andy Reaser
9698564,Julie Wong
2528015,Jase Miles-Perez
2715597,Meg Marinis
...,...
0012182,Jim Adler
7258376,Nathalie Simone Touboul
14153017,Jeff Qiu


In [362]:
writing_creds_df

,writer_id
ep_id,
0592902,0722274
0592923,0722274
0592926,0722274
0592915,0722274
0592919,0722274
...,...
27190233,1068424
27190233,2675977
27190233,4784537


In [388]:
writers_df.drop_duplicates(inplace=True)

## Export

In [363]:
episodes_df.to_csv("data/episodes.csv")

In [389]:
writers_df.to_csv("data/writers.csv")
writing_creds_df.to_csv("data/writing_credits.csv")